## 加载数据

In [1]:
import numpy as np
train_images,train_labels = None,None
test_images,test_labels = None,None
try:
    train_images = np.load('datasets/mnist_npy/train_images.npy')
    train_labels = np.load('datasets/mnist_npy/train_labels.npy')
    test_images = np.load('datasets/mnist_npy/test_images.npy')
    test_labels = np.load('datasets/mnist_npy/test_labels.npy')
except:
    print('Load failed')

In [2]:
print('train data shape:{}\ntest data shape:{}'.format(train_images.shape,test_images.shape))

train data shape:(60000, 28, 28)
test data shape:(10000, 28, 28)


## 数据预处理

In [3]:
import torch
import torch.nn as nn
train_images = torch.tensor(train_images/255.)
train_labels = torch.tensor(train_labels)
test_images = torch.tensor(test_images/255.)
test_labels = torch.tensor(test_labels)


In [4]:
train_images.size()

torch.Size([60000, 28, 28])

In [5]:
X_train = torch.unsqueeze(train_images,dim=1).type(torch.FloatTensor)
y_train = train_labels.type(torch.LongTensor)
X_test = torch.unsqueeze(test_images,dim=1).type(torch.FloatTensor)
y_test = test_labels.type(torch.LongTensor)

print(X_train.size())
print(y_train.size())

torch.Size([60000, 1, 28, 28])
torch.Size([60000])


## 定义网络结构

In [8]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.out = nn.Linear(in_features=16*12*12,out_features=10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = x.view(x.size(0),-1)
        output = self.out(x)
        return output
    

In [9]:
cnn = MyCNN()
print(cnn)

MyCNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (out): Linear(in_features=2304, out_features=10, bias=True)
)


## 训练

In [11]:
optimizer = torch.optim.Adam(cnn.parameters(),lr=0.001)
loss_func = nn.CrossEntropyLoss()

batch_size = 50
for step in range(1000):
    X_batch = X_train[step*batch_size:(step+1)*batch_size]
    y_batch = y_train[step*batch_size:(step+1)*batch_size]
    output = cnn(X_batch)
    loss = loss_func(output,y_batch)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    if step%100 == 0:
        print('[{0}/{1}({2:.0f})%]\t Loss:{3:.6f}'.format(
                            batch_size*step,
                            len(X_train),
                            (batch_size*step*100.0)/len(X_train),loss))

[0/60000(0)%]	 Loss:0.128275
[5000/60000(8)%]	 Loss:0.116906
[10000/60000(17)%]	 Loss:0.109363
[15000/60000(25)%]	 Loss:0.016473
[20000/60000(33)%]	 Loss:0.129094
[25000/60000(42)%]	 Loss:0.011980
[30000/60000(50)%]	 Loss:0.170794
[35000/60000(58)%]	 Loss:0.038379
[40000/60000(67)%]	 Loss:0.077099
[45000/60000(75)%]	 Loss:0.068887


## 测试

In [12]:
test_number = 1000
test_data = X_test[:test_number]
test_target = y_test[:test_number]
test_output = cnn(test_data)
test_loss = loss_func(test_output,test_target)

print(test_output.size())

predict = test_output.argmax(dim=1,keepdim=True)
correct = predict.eq(test_target.view_as(predict)).sum().item()
accuracy = 100.0 * correct/test_number
print('Loss:{0:.4f},Accuracy:{1}/{2}({3:.4f})'.format(
                test_loss,correct,test_number,accuracy))

torch.Size([1000, 10])
Loss:0.0708,Accuracy:978/1000(97.8000)


In [15]:
predict.size()

torch.Size([1000, 1])